In [1]:
import hail as hl
hl.init(app_name="QC")

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/eriurn/submit/python37_intel/lib/python3.7/site-packages/pyspark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/hdp/3.1.0.0-78/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-03-14 11:25:08 WARN  Hail:43 - This Hail JAR was compiled for Spark 3.1.1, running with Spark 3.1.2.
  Compatibility is not guaranteed.
2022-03-14 11:25:12 WARN  Client:69 - Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


Running on Apache Spark version 3.1.2
SparkUI available at http://brenner-fpga-mlx.mlx:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.68-13190f0b6103
LOGGING: writing to /home/eriurn/Hail-joint-caller/notebooks/hail-20220314-1124-0.2.68-13190f0b6103.log


In [2]:
mt_path = "file:///directflow/ClinicalGenomicsPipeline/dev/2021-02-04-PIPELINE-1885-All-Hail/EricData/Glucoma/gvcf_WES.combined.VQSR.filtered.VEP.final.mt/"

In [3]:
mt = hl.read_matrix_table(mt_path)

In [23]:
mt_GT = mt.transmute_entries(GT = hl.experimental.lgt_to_gt(mt.LGT, mt.LA))

In [ ]:
mt_GT = hl.split_multi_hts(mt_GT)

In [ ]:
mt_GT = hl.sample_qc(mt_GT)

In [ ]:
mt_GT = hl.variant_qc(mt_GT)

In [ ]:
impute_sex_mt = hl.impute_sex(mt_GT.GT)

In [ ]:
impute_sex_mt.describe()

In [ ]:
final_mt = mt_GT.annotate_cols(impute_sex = impute_sex_mt[mt_GT.s])

In [ ]:
output_path = "file:///directflow/ClinicalGenomicsPipeline/dev/2021-02-04-PIPELINE-1885-All-Hail/EricData/Glucoma/gvcf_WES.combined.VQSR.filtered.VEP.QC.sex.final.mt/"
final_mt.write(output_path,overwrite=True)

In [ ]:
tmp_mt = mt_GT.annotate_cols(**impute_sex_mt[mt_GT.s])

In [ ]:
final_mt.describe()

In [ ]:
tmp_mt = final_mt.drop(final_mt.gvcf_info)

In [ ]:
hl.export_vcf(tmp_mt, 'file:///directflow/ClinicalGenomicsPipeline/dev/2021-02-04-PIPELINE-1885-All-Hail/EricData/Glucoma/gvcf_WES.combined.VQSR.filtered.VEP.QC.sex.final.vcf')